In [2]:
import pandas as pd
import requests
import urllib.parse
import folium
from folium.plugins import HeatMap

# Read the locations.csv file using a relative path, skipping bad lines
locations_df = pd.read_csv('../text_structuration/script/outfile_corrected.csv', on_bad_lines='skip')

# Function to get latitude and longitude using MediaWiki API
def get_lat_long(city):
    try:
        # Encode the city name for the URL
        title = urllib.parse.quote(city)
        
        # MediaWiki API to get the coordinates
        wiki_api_url = f"https://de.wikipedia.org/w/api.php?action=query&prop=coordinates&titles={title}&format=json"
        response = requests.get(wiki_api_url).json()
        
        # Extract the coordinates from the response
        pages = response['query']['pages']
        for page_id in pages:
            if 'coordinates' in pages[page_id]:
                coords = pages[page_id]['coordinates'][0]
                return coords['lat'], coords['lon']
            else:
                return None, None
    except requests.RequestException:
        return None, None

# Count occurrences of each city
city_counts = locations_df['Place_of_Origin'].value_counts().reset_index()
city_counts.columns = ['Place_of_Origin', 'Count']

# Add coordinates for each city
city_counts['Coordinates'] = city_counts['Place_of_Origin'].apply(get_lat_long)
city_counts[['Latitude', 'Longitude']] = pd.DataFrame(city_counts['Coordinates'].tolist(), index=city_counts.index)

# Remove rows with None coordinates
city_counts = city_counts.dropna(subset=['Latitude', 'Longitude'])

# Get coordinates for Zurich
zurich_coords = get_lat_long('Zurich')

# Read the Hotels.csv file using a relative path, skipping bad lines
hotels_df = pd.read_csv('../text_structuration/script/hotel_geoloc_id.csv', on_bad_lines='skip')

# Convert 'lat' and 'lon' columns to numerical values
hotels_df['lat'] = hotels_df['lat'].str.replace(',', '.').astype(float)
hotels_df['lon'] = hotels_df['lon'].str.replace(',', '.').astype(float)

# Clean the data by removing rows with NaN values in 'lat' or 'lon' columns
hotels_df = hotels_df.dropna(subset=['lat', 'lon'])

# Create a Folium map centered around Switzerland with the OpenStreetMap tile layer
switzerland_map = folium.Map(location=[46.8, 8.33], zoom_start=4, tiles="OpenStreetMap")

# Add the historical map tile layer for Switzerland as an overlay
tiles = "https://wmts.geo.admin.ch/1.0.0/ch.swisstopo.zeitreihen/default/18901231/3857/{z}/{x}/{y}.png"
historical_layer = folium.TileLayer(tiles=tiles, name="swiss 1890", attr="swisstopo 1892", control=True, overlay=True)
historical_layer.add_to(switzerland_map)

# Prepare data for the heat map
heat_data = [[row['Latitude'], row['Longitude'], row['Count']] for _, row in city_counts.iterrows()]

# Add heat map layer
HeatMap(heat_data).add_to(switzerland_map)
'''
# Add markers for each city with a size relative to the number of occurrences
for _, row in city_counts.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Count'] * 3,  # Scale the size by the count
        popup=f"{row['Place_of_Origin']} (Count: {row['Count']})",
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(switzerland_map)

    # Draw a line from the city to Zurich
    #folium.PolyLine(
    #    locations=[[row['Latitude'], row['Longitude']], zurich_coords],
    #    color='green',
    #    weight=2,
    #    opacity=0.7
    #).add_to(switzerland_map)
    
'''
# Add markers for hotels from the Hotels.csv data
for _, hotel in hotels_df.iterrows():
    folium.Marker(
        location=[hotel['lat'], hotel['lon']],
        popup=hotel['Hotel'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(switzerland_map)

# Add layer control to toggle between layers
folium.LayerControl().add_to(switzerland_map)

# Save map to an HTML file
switzerland_map.save("switzerland_map.html")

# If you're running this in a Jupyter environment, display the map
switzerland_map